In [ ]:
!tar -zxf ../input/pawpularity-ml-model/wheelhouse.tar.gzy
!pip install -q -r wheelhouse/requirements.txt --no-index --find-links wheelhouse
!rm -r wheelhouse

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
from fastai.vision.all import *
from pycaret.regression import *
from tqdm import tqdm

In [ ]:
test_path = Path('../input/petfinder-pawpularity-score/test')
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df['path'] = test_df['Id'].apply(lambda x: test_path/(x+'.jpg'))
test_df.tail()

In [ ]:
BS = 4 if len(test_df) == 8 else 64
dls = DataBlock(
    (ImageBlock()),
    get_x = ColReader('path'),
    item_tfms = Resize(224)
).dataloaders(test_df, bs = BS)
dls.show_batch()

In [ ]:
dl = dls.test_dl(test_df)

In [ ]:
all_preds = []
ml_model = load_model(f'../input/pawpularity-ml-model/final_ml_model')
nn_model = timm.create_model('swin_large_patch4_window7_224', pretrained=False, num_classes=1)

for FOLD in range(10):

    nn_model.load_state_dict(torch.load(f'../input/pawpularity-swim-models-10-fold/models/model_f{FOLD}.pth'))
    if torch.cuda.is_available():    
        nn_model.cuda()
    nn_model.eval();

    ## NN Model predictions
    all_feats = []
    all_nn_preds = []
    for xb in tqdm(dl):
        with torch.no_grad():
            feats = nn_model.forward_features(xb[0])
            preds = nn_model.head(feats)        
        all_feats.append(feats.sigmoid().cpu().numpy())
        all_nn_preds.append(preds.sigmoid().cpu().numpy())
    all_feats = np.vstack(all_feats)
    all_nn_preds = np.vstack(all_nn_preds)

    ## ML Model predictions
    train_emb_df = pd.DataFrame(all_feats, columns = [f'x_{x}' for x in range(all_feats.shape[1])])
    all_ml_preds = predict_model(ml_model, data = train_emb_df)
    fold_df = pd.DataFrame({
        'Id': test_df['Id'],
        'nn_preds': all_nn_preds.flatten() * 100,
        'ml_preds': all_ml_preds['Label']
    })
    all_preds.append(fold_df)

all_preds = pd.concat(all_preds).groupby('Id').mean().reset_index()

In [ ]:
all_preds['Pawpularity'] = (all_preds['nn_preds'] + all_preds['ml_preds'])/2
all_preds[['Id', 'Pawpularity']].to_csv('submission.csv', index = False)
all_preds

In [ ]:
!rm logs.log